## Логика игры

Приветствие и правила игры

Показ игровой доски ИИ (все в нулях)

Создание рандомных кораблей на доске ИИ

Запрос создания кораблей Игрока (показ при каждом размещении кораблей)

### Loop:
Ход игрока

Проверка правильности хода

Размещение на доске ИИ и отображение его доски

Проверка на ПОБЕДУ

Ход ИИ и отображение доски Игрока

Проверка на ПОБЕДУ


# ...Error Classes

In [76]:
class BoardOutError(Exception):
    def __init__(self,value):
        self.value = value
    
    def __str__(self):
        return f"{self.value} лежит за пределами игрового поля. Введите двузначное число, каждая цифра которого от 1 до 6, включительно."

class NonEmptyError(Exception):
    def __init__(self,value):
        self.value = value
    
    def __str__(self):
        return f"Выбраная ячейка - {self.value} - непустая. Стрелять можно только в пустые ячейки."

class ShipLocationError(Exception):
    def __init__(self,length, front, direction):
        self.length = length
        self.front = front
        self.direction = direction
    
    def __str__(self):
        self.front = (self.front[0] + 1, self.front[1] + 1)
        direction_word = {
            1: 'Вправо',
            2: 'Вниз'
        }
        return f"Указанные параметры - длина ({self.length}), положение носа корабля {self.front} и направление ({direction_word[self.direction]}) не позволяют корректно разместить корабль на игровом поле."
    

# ...Board, Ship, Dot Classes

In [77]:
class Dot:
    def __init__(self, row, col):
        self.row = row
        self.col = col
        
    def __eq__(self, other):
        return self.row == other.row and self.col == other.col
    
    @property
    def show(self):
        return(self.row, self.col)

In [78]:
grid = [['o' for i in range(6)] for i in range(6)]

def print_grid(L):
    print(*[" ", *[i for i in range(1,7)]], sep=' | ')
    for i in range(6):
        print(i+1, *L[i], sep=' | ')
    return

In [99]:
class Ship:
    def __init__(self, length, front, direction, life):
        self.length = length
        self.front = front
        self.direction = direction
        self.life = life
    
    def dots(self):
        ship_cells = []
        direction_vector = {
            1:  (0, 1),
            2:  (1, 0)
        }
                
        for i in range(self.length):
            cell = (self.front[0] + direction_vector[self.direction][0]*(i), \
                    self.front[1] + direction_vector[self.direction][1]*(i))
            ship_cells.append(cell)
        return ship_cells


In [100]:
ship_1 = Ship(3,(0,2),1,3)
ship_1.dots()



[(0, 2), (0, 3), (0, 4)]

In [100]:
a = [(1, 3), (1, 4), (1, 3), (1, 4)]
print(set(a))


{(1, 3), (1, 4)}


In [206]:
class Board:
    POSSIBLE_CELLS = []
    for i in range(6):
        for j in range(6):
            POSSIBLE_CELLS.append((i,j))
#     print(POSSIBLE_CELLS)
    
    player_shots_left = []
    for i in POSSIBLE_CELLS:
        player_shots_left.append(Dot(*i))
#     print(len(player_shots_left))
    
    
    start_cells_conditions = [['o' for i in range(6)] for i in range(6)]
       
    __s = u'\u220E'     #код квадрата в системе unicode 
    
    def __init__(self, hid, cells_conditions = start_cells_conditions, ships = [], alive_ships = 7):
        self.cells_conditions = cells_conditions    # list
        self.ships = ships                          # list with all ships
        self.hid = hid                              # bool
        self.alive_ships = alive_ships              # int

        
    def add_ship(self, ship):      #ставит корабль на доску (если ставить не получается, выбрасываем исключения)       
        for i in range(len(ship.dots())):
            if ship.dots()[i] in self.contoured_ships():
                raise ShipLocationError(ship.length, ship.front, ship.direction)
        self.ships.append(ship.dots())
        print(f'ships = {self.ships}')
            
    
    def contoured_ships(self):      # точки всех кораблей + их контуров
        
        existing_ships_dots = []
        for i in range(len(self.ships)):
            existing_ships_dots += self.ships[i]
        
        contoured_ships_dots = []
        for dot in range(len(existing_ships_dots)):
            for i in [-1, 0, 1]:
                for j in [-1, 0, 1]:
                    tested_dot = (existing_ships_dots[dot][0] + i, existing_ships_dots[dot][1] + j)
                    if tested_dot in self.POSSIBLE_CELLS:
                        contoured_ships_dots.append(tested_dot)
        return list(set(contoured_ships_dots))
                
        
    def show_board(self):      #выводит доску в консоль в зависимости от параметра hid.
        ships_gui = self.start_cells_conditions.copy()
        
        existing_ships_dots = []
        for i in range(len(self.ships)):
            existing_ships_dots += self.ships[i]
        
        for i in existing_ships_dots:
            ships_gui[i[0]][i[1]] = self.__s
        
        if not hid:
            return self.gui_print(ships_gui)
    
    def out(self, dot):      #для точки (объекта класса Dot) возвращает True, если точка выходит за пределы поля
        print(111)
        if dot in self.POSSIBLE_CELLS:
            return False
        else:
            return True
    
    def shot(self):         #делает выстрел по доске (если есть попытка выстрелить за \
                            #пределы и в использованную точку, нужно выбрасывать исключения
        while True:         #Loop для получения правильного ввода от Игрока

            try:
                player_shot = int(input("В какую ячейку будем стрелять?"))
                row = player_shot // 10 - 1
                col = player_shot % 10 - 1
            except Exception:
                print('Необходимо ввести двузначное число!')
                break
            
            if self.out((row, col)):
                raise BoardOutError(player_shot)
            else:
                shot = Dot(row, col)

                if shot not in self.player_shots_left:
                    err = NonEmptyError(player_shot)
                    print(err)
                else:
                    existing_ships_dots = []
                    for i in range(len(self.ships)):
                        existing_ships_dots += self.ships[i]
                    if (row, col) in existing_ships_dots:
                        self.cells_conditions[row][col] = 'X'
                        self.player_shots_left.remove(Dot(row,col))
                        print(self.cells_conditions)    
                    else:

                        self.cells_conditions[row][col] = 'T'
                        self.player_shots_left.remove(Dot(row,col))
                        print(self.cells_conditions)
                    print('Ход компьютера:')
                    break

            
    def gui_print(self, L):
        print(*[" ", *[i for i in range(1,7)]], sep=' | ')
        for i in range(6):
            print(i+1, *L[i], sep=' | ')
        return

In [208]:
player = 'Player'
hid = True if player == 'AI' else False

ships = [[(0,0)],
         [(2,2),(2,3)],
         [(4,0),(4,1),(4,2)]
        ]
# ships = []
AI_board = Board(hid, ships = ships)
AI_board.add_ship(ship_1)
print(len(AI_board.contoured_ships()))

AI_board.shot()

ships = [[(0, 0)], [(2, 2), (2, 3)], [(4, 0), (4, 1), (4, 2)], [(0, 2), (0, 3), (0, 4)]]
29
В какую ячейку будем стрелять?11
111
Выбраная ячейка - 11 - непустая. Стрелять можно только в пустые ячейки.
В какую ячейку будем стрелять?12
111


IndexError: list index out of range

In [184]:
AI_board.show_board()

  | 1 | 2 | 3 | 4 | 5 | 6
1 | ∎ | o | ∎ | ∎ | ∎ | o
2 | o | o | o | o | o | o
3 | o | o | ∎ | ∎ | o | o
4 | o | o | o | o | o | o
5 | ∎ | ∎ | ∎ | o | o | o
6 | o | o | o | o | o | o


In [188]:
AI_board.shot()
print(len(AI_board.player_shots_left))

В какую ячейку будем стрелять?12
Необходимо ввести двузначное число!
В какую ячейку будем стрелять?13
Необходимо ввести двузначное число!


KeyboardInterrupt: Interrupted by user

In [104]:
ship_2 = Ship(2,(3,5),2,2)
ship_2.dots()

AI_board.add_ship(ship_2)

ships = [[(0, 2), (0, 3), (0, 4)], [(3, 5), (4, 5)]]


In [128]:
start_cells_conditions = [['o' for i in range(6)] for i in range(6)]
print(start_cells_conditions)

[['o', 'o', 'o', 'o', 'o', 'o'], ['o', 'o', 'o', 'o', 'o', 'o'], ['o', 'o', 'o', 'o', 'o', 'o'], ['o', 'o', 'o', 'o', 'o', 'o'], ['o', 'o', 'o', 'o', 'o', 'o'], ['o', 'o', 'o', 'o', 'o', 'o']]


In [ ]:
ship_1 = Ship(3,(2,2),(0,1),3)
print(ship_1)
ship_1_cells = ship_1.dots()
print(ship_1_cells)

for i in range(len(ship_1_cells)):
    row = ship_1_cells[i][0]
    col = ship_1_cells[i][1]
    print(row, col)
    
    grid[row-1][col-1] = "X"

print_grid(grid)

In [42]:
print(sorted(AI_board.contoured_ships()))

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4)]


In [ ]:
while True:      #Loop для получения правильного ввода от Игрока
    try:
        user_shot = int(input("В какую ячейку будем стрелять?"))
        row = user_shot // 10
        col = user_shot % 10

        if row not in range(1,7) or col not in range(1,7):
            err = BoardOutError(user_shot)
            print(err)

        else:
            shot = Dot(row,col)
            if shot in user_shots:
                err = NonEmptyError(user_shot)
                print(err)
            else:
                grid[row-1][col-1] = 'X'
                user_shots.append(shot)
                print_grid(grid)
                print('Ход компьютера:')
                break

    except Exception:
        print('Необходимо ввести двузначное число!')

In [ ]:
for i in range(len(user_shots)):
    print(i, user_shots[i].rtrn())
user_shots[2] == user_shots[2]


In [ ]:
# from random import randint

# while True:
#     row, col = randint(1,6),randint(1,6)
#     print(row,col)
#     shot = Dot(row, col)
#     if shot not in PC_shots:
#         grid[row-1][col-1] = 'X'
#         PC_shots.append(shot)
#         print_grid(grid)
#         break

unused_shots = []
for i in range(6):
    for j in range(6):
        unused_shots.append((i,j))
                               
                               
print(unused_shots[randint(1,len(unused_shots))])


In [ ]:
s = u'\u220E'     #код квадрата в системе unicode 


In [110]:
class Test:
    a = []
    __hid = 123
    def __init__(self, c):
        self.c = c
    
    def change_a(self, aa):
        self.a.append(aa*self.__hid)
        return(self.a)


In [113]:
test = Test(10)
print(test.change_a(10))

[615, 1230]


In [47]:
test = Test(12)
test.change_a(2)

[246, 246, 246]

In [40]:
print(test.c)
test.c = 500
print(test.c)

345
500


In [162]:
grid = [['o' for i in range(6)] for i in range(6)]

def print_grid(L):
    print(*[" ", *[i for i in range(1,7)]], sep=' | ')
    for i in range(6):
        print(i+1, *L[i], sep=' | ')
    return

# print_grid(grid)

us_in = 55
row = us_in // 10
col = us_in % 10
grid[row-1][col-1] = "X"
print_grid(grid)

grid_user = grid.copy()
grid[3][3] = "X"
print_grid(grid_user)
print(id(grid_user))
print(id(grid))

  | 1 | 2 | 3 | 4 | 5 | 6
1 | o | o | o | o | o | o
2 | o | o | o | o | o | o
3 | o | o | o | o | o | o
4 | o | o | o | o | o | o
5 | o | o | o | o | X | o
6 | o | o | o | o | o | o
  | 1 | 2 | 3 | 4 | 5 | 6
1 | o | o | o | o | o | o
2 | o | o | o | o | o | o
3 | o | o | o | o | o | o
4 | o | o | o | X | o | o
5 | o | o | o | o | X | o
6 | o | o | o | o | o | o
3067817364736
3067817185984


In [ ]:
print(user_shots)

In [ ]:
user_shots[0] == user_shots[1]

In [ ]:
# Python program showing the use of
# @property
  
class Geeks:
     def __init__(self):
          self._age = 0
       
     # using property decorator
     # a getter function
     @property
     def age(self):
         print("getter method called")
         return self._age
       
     # a setter function
     @age.setter
     def age(self, a):
         if(a < 18):
            err = ValueError("Sorry you age is below eligibility criteria")
            print(err)
         print("setter method called")
         self._age = a
  
mark = Geeks()
print(mark.age)
mark.age = int(input())
  
print(mark.age)

# ВНЕШНЯЯ ЛОГИКА

In [ ]:
class Player:
    def __init__(self, )
    
    @property
    def my_board(self):
        self.xxx = Board(grid_ships)
    
    @property
    def opponent_board(self):
        self.xxx = Board(grid_ships)
        
    def ask(self):
        pass
    
    def move(self):
        while True:
            self.Player.ask()
            
            